In [111]:
#Importing the libraries
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [112]:
#Read csv file from German to English Language
cars=pd.read_csv('autos.csv',encoding = "ISO-8859-1" )

In [113]:
#Shape of the data
cars.shape

(371528, 20)

In [114]:
#Renaming the column names

cars_col = ['Date_Crawled','Name','Seller','Offer_Type','Price','Ab_test','Vehicle_Type',
           'Year_of_Registration','Gear_box','Power_ps','Model','Kilometer','Month_of_Registration',
           'Fuel_Type','Brand','Not_repaired_damage','Date_Created','No_of_pictures','Postal_Code',
           'Last_Seen']
cars.columns = cars_col

In [115]:
#Drop a column after observing the record difference
#As we can see its a biased data and model will only learn which has the largest value
cars.Seller.value_counts()

privat        371525
gewerblich         3
Name: Seller, dtype: int64

In [116]:
#Now try for offer Type and observe the record difference
cars.Offer_Type.value_counts()

Angebot    371516
Gesuch         12
Name: Offer_Type, dtype: int64

In [117]:
#Now try for Ab_test and observe the record difference
#Here the values are not biased so this will defnitely play an important role in analysis
cars.Ab_test.value_counts()

test       192585
control    178943
Name: Ab_test, dtype: int64

In [118]:
#Now try for Vehicle_Type and observe the record difference
#Here the values are not biased so this is also an important variable for the prediction
cars.Vehicle_Type.value_counts()

limousine     95894
kleinwagen    80023
kombi         67564
bus           30201
cabrio        22898
coupe         19015
suv           14707
andere         3357
Name: Vehicle_Type, dtype: int64

In [119]:
#dropping the following columns
cars.drop(['Seller','Offer_Type','No_of_pictures','Name','Date_Crawled','Date_Created','Last_Seen',
           'Postal_Code','Kilometer'],axis=1, inplace=True)

In [120]:
cars.shape

(371528, 11)

In [121]:
#To check what all the variables have null values
cars.isnull().sum()

Price                        0
Ab_test                      0
Vehicle_Type             37869
Year_of_Registration         0
Gear_box                 20209
Power_ps                     0
Model                    20484
Month_of_Registration        0
Fuel_Type                33386
Brand                        0
Not_repaired_damage      72060
dtype: int64

In [122]:
#Now try for Gear_box and observe the record difference
#Here the values are not biased so this can be an important variable
cars.Gear_box.value_counts()

manuell      274214
automatik     77105
Name: Gear_box, dtype: int64

In [123]:
#Grouping the Brand and based on the gear count will check the gear type with max values
cars.groupby("Brand")["Gear_box"].value_counts().head()

Brand       Gear_box 
alfa_romeo  manuell       2064
            automatik      146
audi        manuell      20841
            automatik    10698
bmw         manuell      25323
Name: Gear_box, dtype: int64

In [124]:
#To fill the missing data of gear box by most occuring geartype with respect to its brand
groups_max=dict()
Brand_names= cars.Brand.unique()
Brand_names

#Through for loop fill the values of the gear box w.r.t to the Brand names
for i in Brand_names:
    gear_values=cars[cars.Brand==i]['Gear_box'].value_counts()
    groups_max[i]=gear_values.index[0]

In [125]:
#Now check if the location of the brand name and gearbox value is null then fill the 
#value with the max count
for i in Brand_names:
    cars.loc[(cars.Brand == i) & (cars.Gear_box.isnull()) ,"Gear_box" ] = groups_max[i]

In [126]:
#check if gear box has any missing values
cars.Gear_box.isnull().sum()

0

In [127]:
#Fill the NA values for the Fuel_type variable with most frequent values
cars.Fuel_Type.value_counts()

#As benzin is most frequently occured filling the NA values with 'benzin'
cars['Fuel_Type'].fillna('benzin',inplace=True)

#Check if any null value in present in Fuel type
cars.Fuel_Type.isnull().sum()

0

In [128]:
#Fill the NA values for the Not_repaired_damage variables with most frquent values
cars.Not_repaired_damage.value_counts()

#As nein is most frequently occured filling the NA values with 'nein'
cars['Not_repaired_damage'].fillna('nein',inplace=True)

#Check if any null value in present in Fuel type
cars.Not_repaired_damage .isnull().sum()

0

In [129]:
#Fill the NA values for the Model variable with most frquent values
cars.Model.value_counts()

#As golf is most frequently occured filling the NA values with 'golf'
cars['Model'].fillna('golf',inplace=True)

#Check if any null value in present in Fuel type
cars.Model.isnull().sum()

0

In [130]:
#Its time to fill the vehicle type before that lets check the unique values 
cars.Vehicle_Type.unique()

array([nan, 'coupe', 'suv', 'kleinwagen', 'limousine', 'cabrio', 'bus',
       'kombi', 'andere'], dtype=object)

In [131]:
#Fill the values with the same logic implemented for the gear type
#Based on the Fuel Type we will fill the vehicle type with the max value counts
group_max = dict()
fuel_names = cars.Fuel_Type.unique()
fuel_names

for i in fuel_names:
    vehicle_val = cars[cars.Fuel_Type==i]['Vehicle_Type'].value_counts()
    group_max[i]=vehicle_val.index[0]

In [132]:
for i in fuel_names:
    cars.loc[(cars.Fuel_Type==i) & (cars.Vehicle_Type.isnull()),"Vehicle_Type"] = group_max[i]

In [133]:
#Check for all the variables if it is null
cars.isnull().sum()

Price                    0
Ab_test                  0
Vehicle_Type             0
Year_of_Registration     0
Gear_box                 0
Power_ps                 0
Model                    0
Month_of_Registration    0
Fuel_Type                0
Brand                    0
Not_repaired_damage      0
dtype: int64

In [134]:
#Dimensions of the dataframe
cars.shape

(371528, 11)

In [135]:
#First few records of the dataframe
cars.head()

,Price,Ab_test,Vehicle_Type,Year_of_Registration,Gear_box,Power_ps,Model,Month_of_Registration,Fuel_Type,Brand,Not_repaired_damage
0,480,test,kleinwagen,1993,manuell,0,golf,0,benzin,volkswagen,nein
1,18300,test,coupe,2011,manuell,190,golf,5,diesel,audi,ja
2,9800,test,suv,2004,automatik,163,grand,8,diesel,jeep,nein
3,1500,test,kleinwagen,2001,manuell,75,golf,6,benzin,volkswagen,nein
4,3600,test,kleinwagen,2008,manuell,69,fabia,7,diesel,skoda,nein


In [136]:
#Filling the categorical variable through get_dummies
cars_dummies = pd.get_dummies(cars, columns=['Ab_test','Vehicle_Type','Gear_box','Model',
                                             'Fuel_Type','Brand','Not_repaired_damage'],drop_first=True)
cars_dummies.head()

,Price,Year_of_Registration,Power_ps,Month_of_Registration,Ab_test_test,Vehicle_Type_bus,Vehicle_Type_cabrio,Vehicle_Type_coupe,Vehicle_Type_kleinwagen,Vehicle_Type_kombi,...,Brand_skoda,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,Not_repaired_damage_nein
0,480,1993,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
1,18300,2011,190,5,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9800,2004,163,8,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1500,2001,75,6,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
4,3600,2008,69,7,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1


In [137]:
#Filtering the outliers from the files before that first check the year of registration and Prices values
cars_final = cars_dummies[(cars_dummies.Year_of_Registration < 2020)  & (cars_dummies.Year_of_Registration > 1947)]
cars_final = cars_dummies[(cars_dummies.Price > 500) & (cars_dummies.Price < 150000)]

In [138]:
cars_final.head()

,Price,Year_of_Registration,Power_ps,Month_of_Registration,Ab_test_test,Vehicle_Type_bus,Vehicle_Type_cabrio,Vehicle_Type_coupe,Vehicle_Type_kleinwagen,Vehicle_Type_kombi,...,Brand_skoda,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,Not_repaired_damage_nein
1,18300,2011,190,5,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9800,2004,163,8,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1500,2001,75,6,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
4,3600,2008,69,7,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
5,650,1995,102,10,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [150]:
#Splitting the data into train and test
cars_final_X = cars_final.iloc[:,1:].values
cars_final_y = cars_final.iloc[:,0].values

X_train, X_test_val, y_train, y_test_val = train_test_split(cars_final_X, cars_final_y, test_size=0.2)

sc=StandardScaler()
X_train= sc.fit_transform(X_train)
X_test_val = sc.transform(X_test_val)

C:\Users\Aish\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Aish\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Aish\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [151]:
#Implementing the linear regression
lr=LinearRegression()
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [152]:
#Prediction of the values
y_pred=lr.predict(X_test_val)
y_pred[0:5]

array([3809.74921599, 3130.65460083, 5267.58934992, 5871.29696536,
       2408.9217827 ])

In [153]:
#Checking with the actual results
y_test_val[0:5]

array([3990, 3199, 3100, 5850, 3900], dtype=int64)